# Mimic `data.TrialHandler`

In [2]:
import numpy as np

In [7]:
def _makeIndices(inputArray):
    """
    Creates an array of tuples the same shape as the input array
    where each tuple contains the indices to itself in the array.

    Useful for shuffling and then using as a reference.
    """
    # make sure its an array of objects (can be strings etc)
    inputArray = np.asarray(inputArray, 'O')
    
    # get some simple variables for later
    dims = inputArray.shape  # shape of inputArray
    dimsProd = np.product(dims)  # number of items
    dimsN = len(dims)  # number of axes / dimensions
    dimsList = list(range(dimsN))  # [0, 1, ..., dimsN - 1]
    
    # this creates space for an array of any objects
    arrayOfTuples = np.ones(dimsProd, 'O')

    # for each dimension create list of its indices (using modulo)
    listOfLists = []
    for thisDim in dimsList:  
        prevDimsProd = np.product(dims[:thisDim])  # ??
        thisDimVals = np.arange(dimsProd) / prevDimsProd % dims[thisDim]
        listOfLists.append(thisDimVals)

    # convert to array
    indexArr = np.asarray(listOfLists)
    for n in range(dimsProd):
        arrayOfTuples[n] = tuple((indexArr[:, n]))
    return (np.reshape(arrayOfTuples, dims)).tolist()

In [78]:
trialList = [{'condition':'a'}, {'condition':'b'}, {'condition':'c'}]

In [79]:
indices = _makeIndices(trialList)

## Mimic `_createSequence`

In [80]:
def shuffleArray(array, seed):
    array_to_shuffle = array.copy()
    np.random.seed(seed)
    np.random.shuffle(array_to_shuffle)
    return array_to_shuffle

In [81]:
method = 'random'
nReps = 5
seed = 42

In [82]:
indices = np.asarray(indices, dtype=int)

if method == 'random':
    sequenceIndices = []
    for thisRep in range(nReps):
        thisRepSeq = shuffleArray(indices.flat, seed=seed).tolist()
        seed = None  # so that we only seed the first pass through!
        sequenceIndices.append(thisRepSeq)
    sequenceIndices = np.transpose(sequenceIndices)

elif method == 'sequential':
    sequenceIndices = np.repeat(indices, self.nReps, 1)

elif method == 'fullRandom':
    # indices*nReps, flatten, shuffle, unflatten; only use seed once
    sequential = np.repeat(indices, self.nReps, 1)  # = sequential
    randomFlat = shuffleArray(sequential.flat, seed=self.seed)
    sequenceIndices = np.reshape(
        randomFlat, (len(indices), self.nReps))

In [83]:
sequenceIndices

array([[0, 0, 0, 2, 2],
       [1, 1, 2, 0, 1],
       [2, 2, 1, 1, 0]])

## Mimic `__next__`

In [84]:
thisTrialN = 0
thisRepN = 0

finished = False
while True:
    
    if thisTrialN == len(trialList):
        # start a new repetition
        thisTrialN = 0
        thisRepN += 1
        print('==========')
        
    if thisRepN >= nReps:
        # all reps complete
        finished = True

    if finished == True:
        break

    thisIndex = sequenceIndices[thisTrialN][thisRepN]
    thisTrial = trialList[thisIndex]
    
    print(thisTrial)
    
    thisTrialN += 1

{'condition': 'a'}
{'condition': 'b'}
{'condition': 'c'}
{'condition': 'a'}
{'condition': 'b'}
{'condition': 'c'}
{'condition': 'a'}
{'condition': 'c'}
{'condition': 'b'}
{'condition': 'c'}
{'condition': 'a'}
{'condition': 'b'}
{'condition': 'c'}
{'condition': 'b'}
{'condition': 'a'}
